## 4.4 train/test 분리 MF 알고리즘

In [48]:
import os
import numpy as np
import pandas as pd

r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv('ml-100k/u.data',
            sep = '\t',
            names = r_cols,
            encoding='latin1')
            
# timestamp 제거
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

# train/test set 분리
from sklearn.utils import shuffle
train_size = 0.75
# (사용자 - 영화 - 평점)
ratubgs = shuffle(ratings,random_state=2021)
cutoff = int(train_size * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

class NEW_MF():
    def __init__(self, ratings, hyper_params):
        self.R = np.array(ratings)
        # 사용자 수(num_users), 아이템 수(num_items)를 받아온다.
        self.num_users, self.num_items = np.shape(self.R)
        # 아래는 MF weight 조절을 위한 하이퍼파라미터이다.
        # K : 잠재요인(latent factor)의 수
        self.K = hyper_params['K']
        # alpha : 학습률
        self.alpha = hyper_params['alpha']
        # beta : 정규화 계수
        self.beta = hyper_params['beta']
        # iterations : SGD의 계산을 할 때 반복 횟수
        self.iterations = hyper_params['iterations']
        # verbose : SGD의 학습 과정을 중간중간에 출력할 것인지에 대한 여부
        self.verbose = hyper_params['verbose']
        
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)

    def rmse(self):
        # self.R에서 평점이 있는(0이 아닌) 요소의 인덱스를 가져온다.
        xs, ys = self.R.nonzero()
        # prediction과 error를 담을 리스트 변수 초기화
        self.predictions = []
        self.errors = []
        
        # 평점이 있는 요소(사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행한다.
        for x,y in zip(xs, ys):
            # 사용자 x, 아이템 y에 대해서 평점 예측치를 get_prediction() 함수를 사용해 계산한다.
            prediction = self.get_prediction(x,y)
            # 예측값을 예측갑 리스트에 추가한다.
            self.predictions.append(prediction)
            # 오차값을 리스트에 추가
            self.errors.append(self.R[x,y] - prediction)
        
        # 예측값 리스트와 오차값 리스트를 numpy array 형태로 변환한다.
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        
        # error를 활용해서 RMSE 도출
        return np.sqrt(np.mean(self.errors ** 2))
    
    def sgd(self):
        for i, j, r in self.samples:
            # 사용자 i, 아이템 j에 대한 평점 예측치 계산
            prediction = self.get_prediction(i,j)
            # 실제 평점과 비교한 오차 계산
            e = (r-prediction)
            
            # 사용자 평가 경향 계산 및 업데이터
            self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
            # 아이템 평가 경향 계산 및 업데이터
            self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

            # P 행렬 계산 및 업데이트
            self.P[i,:] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i,:]))
            # Q 행렬 계산 및 업데이트
            self.Q[j,:] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j,:]))
        
    def get_prediction(self, i, j):
        # 사용자 i, 아이템 j에 대한 평점 예측치를 앞에서 배웠던 식을 이용해서 구한다.
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
        
    # test set 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i,0]]
            y = self.item_id_index[ratings_test.iloc[i,0]]
            z = ratings_test.iloc[i,2]
            test_set.append([x,y,z])
            self.R[x,y] = 0
        self.test_set = test_set
        return test_set
        
    # test set RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2]-predicted, 2)
        return np.sqrt(error/len(self.test_set))
    
    def test(self):
        self.P = np.random.normal(scale = 1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size = (self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        rows, columns = self.R.nonzero()
        self.samples = [(i,j,self.R[i,j])for i,j in zip(rows, columns)]
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print('Iteration : %d ; Train RMSE = %.4f ; test RMSE = %.4f'% (i+1, rmse1, rmse2))
        return training_process
        
    def get_one_prediction(self,user_id,item_id):
        return self.get_prediction(self.user_id_index[user_id],
                                    self.item_id_index[item_id])
                                    
    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)
    
R_temp = ratings.pivot(index = 'user_id',
                        columns='movie_id',
                        values='rating').fillna(0)
                        
hyper_params = {
    'K' : 30, # 잠재요인 변수
    'alpha' : 0.001, # lr
    'beta' : 0.02, # 정규화 개수
    'iterations' : 100, # 반복
    'verbose' : True
}

mf = NEW_MF(R_temp, hyper_params)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration : 10 ; Train RMSE = 0.9586 ; test RMSE = 1.0748
Iteration : 20 ; Train RMSE = 0.9374 ; test RMSE = 1.0943
Iteration : 30 ; Train RMSE = 0.9281 ; test RMSE = 1.1112
Iteration : 40 ; Train RMSE = 0.9226 ; test RMSE = 1.1244
Iteration : 50 ; Train RMSE = 0.9186 ; test RMSE = 1.1347
Iteration : 60 ; Train RMSE = 0.9148 ; test RMSE = 1.1429
Iteration : 70 ; Train RMSE = 0.9106 ; test RMSE = 1.1493
Iteration : 80 ; Train RMSE = 0.9048 ; test RMSE = 1.1547
Iteration : 90 ; Train RMSE = 0.8966 ; test RMSE = 1.1593
Iteration : 100 ; Train RMSE = 0.8853 ; test RMSE = 1.1637


In [49]:
print(mf.full_prediction())

[[3.78270395 3.21656057 3.16850499 ... 3.3415834  3.4518325  3.43572896]
 [3.90592312 3.35096674 3.13393813 ... 3.39080343 3.50857078 3.49420265]
 [3.3531714  2.79977831 2.58888478 ... 2.82167543 2.93662211 2.91091564]
 ...
 [4.23013939 3.64965505 3.42637216 ... 3.65587521 3.77654944 3.74327041]
 [4.3551326  3.75716628 3.56069543 ... 3.80279224 3.92469458 3.8847969 ]
 [3.78232909 3.24660275 2.97042468 ... 3.2496531  3.40118063 3.36613143]]


In [50]:
print(mf.get_one_prediction(1,2))

3.2165605664722543


# 4.5 MF의 최적 파라미터 찾기

In [52]:
results = []
index = []

R_temp = ratings.pivot(index = 'user_id',
                        columns='movie_id',
                        values='rating').fillna(0)

for K in range(50, 261, 10):
    print(f'K : {K}')
    hyper_params = { 
        'K' : K, # 잠재요인 변수
        'alpha' : 0.001, # lr
        'beta' : 0.02, # 정규화 개수
        'iterations' : 300, # 반복
        'verbose' : True
    }
    mf = NEW_MF(R_temp,
        hyper_params)
    test_set = mf.set_test(ratings_test)
    result = mf.test()
    index.append(K)
    results.append(result)

K : 50
Iteration : 10 ; Train RMSE = 0.9588 ; test RMSE = 1.0747
Iteration : 20 ; Train RMSE = 0.9379 ; test RMSE = 1.0942
Iteration : 30 ; Train RMSE = 0.9288 ; test RMSE = 1.1111
Iteration : 40 ; Train RMSE = 0.9236 ; test RMSE = 1.1244
Iteration : 50 ; Train RMSE = 0.9199 ; test RMSE = 1.1347
Iteration : 60 ; Train RMSE = 0.9167 ; test RMSE = 1.1428
Iteration : 70 ; Train RMSE = 0.9133 ; test RMSE = 1.1493
Iteration : 80 ; Train RMSE = 0.9086 ; test RMSE = 1.1547
Iteration : 90 ; Train RMSE = 0.9018 ; test RMSE = 1.1593
Iteration : 100 ; Train RMSE = 0.8921 ; test RMSE = 1.1640
Iteration : 110 ; Train RMSE = 0.8795 ; test RMSE = 1.1690
Iteration : 120 ; Train RMSE = 0.8649 ; test RMSE = 1.1746
Iteration : 130 ; Train RMSE = 0.8492 ; test RMSE = 1.1804
Iteration : 140 ; Train RMSE = 0.8324 ; test RMSE = 1.1854
Iteration : 150 ; Train RMSE = 0.8145 ; test RMSE = 1.1900
Iteration : 160 ; Train RMSE = 0.7954 ; test RMSE = 1.1945
Iteration : 170 ; Train RMSE = 0.7755 ; test RMSE = 1.1989

KeyboardInterrupt: 

In [ ]:
summary = []
for i in range(len(result)):
    RMSE = []
    for result in results[i]:
        RMSE.append(result[2])
    min = np.min(RMSE)
    j = RMSE.index(min)
    summary.append([index[i], j+1, RMSE[j]])

summary